In [ ]:
import numpy as np
import pandas as pd
import pickle

In [ ]:
import sys
print(sys.version)

3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train = pd.read_csv("/content/drive/MyDrive/Filters_colorectal/UNI/train_50.0_uni_selected.csv")
test = pd.read_csv("/content/drive/MyDrive/Filters_colorectal/UNI/test_50.0_uni_selected.csv")

In [ ]:
train.head()

,f_6,f_10,f_12,f_13,f_19,f_20,f_25,f_26,f_31,f_39,...,f_1002,f_1005,f_1008,f_1011,f_1014,f_1015,f_1017,f_1023,f_1024,label
0,3.465876,0.659582,0.000000,6.078495,0.000000,0.0,0.883563,0.0,0.000000,0.0,...,0.0,5.219078,0.000000,0.106189,1.205003,0.0,6.277896,0.0,0.000000,6.0
1,0.000000,0.000000,0.282657,0.000000,1.744669,0.0,0.000000,0.0,1.865544,0.0,...,0.0,0.000000,0.391878,0.000000,0.000000,0.0,0.000000,0.0,10.601711,3.0
2,0.000000,0.000000,1.338476,0.982680,0.000000,0.0,5.016280,0.0,0.000000,0.0,...,0.0,0.000000,2.157906,0.000000,0.000000,0.0,0.000000,0.0,0.000000,5.0
3,0.000000,0.000000,0.297810,0.000000,0.000000,0.0,0.000000,0.0,1.952455,0.0,...,0.0,0.000000,0.484700,0.000000,0.000000,0.0,0.000000,0.0,11.222972,3.0
4,0.000000,0.000000,0.303733,0.000000,0.000000,0.0,0.000000,0.0,1.895066,0.0,...,0.0,0.000000,0.596381,0.000000,0.000000,0.0,0.000000,0.0,9.061059,3.0


In [ ]:
X_train = train.drop(["label"], axis = 1)
X_test = test.drop(["label"], axis = 1)
y_train = train["label"].astype('int32')
y_test = test["label"].astype('int32')

In [ ]:
from sklearn.metrics import accuracy_score, classification_report,f1_score, confusion_matrix, roc_auc_score, matthews_corrcoef

In [ ]:
from sklearn.linear_model import LogisticRegression

#KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(
    n_neighbors=9,
    weights='distance',
    algorithm='auto',
    leaf_size=50
)
knn.fit(X_train, y_train)

KNeighborsClassifier(leaf_size=50, n_neighbors=9, weights='distance')

In [ ]:
with open('knn_model_st.pkl', 'wb') as model_file:
    pickle.dump(knn, model_file)


In [ ]:
from sklearn.svm import SVC

svm = SVC(
    kernel='rbf',
    C=1.5,
    gamma='scale',
    probability=True,
    decision_function_shape='ovr'
)
svm.fit(X_train, y_train)

SVC(C=1.5, probability=True)

In [ ]:
with open('svm_model_st.pkl', 'wb') as model_file:
    pickle.dump(svm, model_file)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(
    n_estimators=1200,
    max_depth=20,
    min_samples_leaf=1,
    random_state=42
)
rfc.fit(X_train, y_train)

RandomForestClassifier(max_depth=20, n_estimators=1200, random_state=42)

In [ ]:
with open('rfc_model_st.pkl', 'wb') as model_file:
    pickle.dump(rfc, model_file)

In [ ]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(
    alpha=0.1,
    max_iter=1000
)
mlp.fit(X_train, y_train)

MLPClassifier(alpha=0.1, max_iter=1000)

In [ ]:
with open('mlp_model_st.pkl', 'wb') as model_file:
    pickle.dump(mlp, model_file)


In [ ]:
probs_model1_val = knn.predict_proba(X_train)
probs_model2_val = svm.predict_proba(X_train)
probs_model3_val = rfc.predict_proba(X_train)
probs_model4_val = mlp.predict_proba(X_train)

In [ ]:
meta_features_train = np.concatenate((probs_model1_val, probs_model2_val, probs_model3_val, probs_model4_val), axis=1)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

probs_model1_test = knn.predict_proba(X_test)
probs_model2_test = svm.predict_proba(X_test)
probs_model3_test = rfc.predict_proba(X_test)
probs_model4_test = mlp.predict_proba(X_test)

meta_features_test = np.concatenate((probs_model1_test, probs_model2_test, probs_model3_test, probs_model4_test), axis=1)


In [ ]:
#Train the meta-model using the meta-features and validation set labels
meta_model = LogisticRegression()

meta_model.fit(meta_features_train, y_train)

# Prediction using the stacked model on new, unseen data

pred = meta_model.predict(meta_features_test)

In [ ]:
#train_pred = meta_model.predict(X_train_base)
test_pred = meta_model.predict(meta_features_test)
test_pred_prob = meta_model.predict_proba(meta_features_test)

auc = roc_auc_score(y_test, test_pred_prob, multi_class='ovr')

conf_matrix = confusion_matrix(y_test, test_pred)

# Compute sensitivity and specificity for each class
sensitivity = conf_matrix.diagonal() / conf_matrix.sum(axis=1)
specificity = []
for i in range(len(conf_matrix)):
    temp = np.delete(conf_matrix, i, 0)
    temp = np.delete(temp, i, 1)
    tn, fp, fn, tp = temp.ravel()[:4]
    specificity.append(tn / (tn + fp))
avg_sensitivity = np.mean(sensitivity)
avg_specificity = np.mean(specificity)


test_acc = accuracy_score(y_test, test_pred)
test_mcc = matthews_corrcoef(y_test, test_pred)
test_f1 = f1_score(y_test, test_pred, average = 'weighted')

print('ACCURACY : %s' % test_acc)
print('MCC : %s' % test_mcc)
print('F1 : %s' % test_f1)
print('AUC: %s' % auc)
print('Sensitivity: %s' % avg_sensitivity)
print('Specivity: %s' %avg_specificity)

In [ ]:
with open('stack.pkl', 'wb') as model_file:
    pickle.dump(meta_model, model_file)
